**Dependencies:**

- To direct c++ std stream to Jupyter notebook:
    `pip install wurlitzer`

- Deformed frame visualization
    Install `meshcat-python`: https://github.com/rdeits/meshcat-python

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np
import pyconmech as cm
from wurlitzer import sys_pipes

import meshcat

import meshcat.geometry as g
from deformed_frame_viz import meshcat_visualize_deformed

In [4]:
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


# check deformation for a completed structure

In [44]:
import os
import time
from deformed_frame_viz import meshcat_visualize_deformed
cwd = os.getcwd()

# json_path = os.path.join(cwd, "assembly_instances/extrusion","fertility.json")
json_path = os.path.join(cwd, "assembly_instances/extrusion","topopt-100_S1_03-14-2019_w_layer.json")

vis.delete()
disc = 10
exagg_ratio = 1.0
time_step = 1.0
scale = 5

with sys_pipes():
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
#     sc.set_output_json(True)
#     sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

    # manually assigned point loads
#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     ext_load[0,4] = -1
#     include_sw = True

    # import load from a json file
#     ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
#     print('external load:')
#     print(ext_load)
#     print('include self weight : {0}'.format(include_sw))
#     sc.set_load(nodal_forces = ext_load)
#     sc.set_self_weight_load(include_sw) # now this is true by default
    
#     sc.set_nodal_displacement_tol(transl_tol=1e-3)
    
    existing_ids = [125, 126, 115, 122, 111, 108, 23, 22, 98, 75, 64, 34, 61, 65, 59, 60, 39, 36, 44, 67]
#     existing_ids = [64] # unsupported
    
#     print("solve: {0}\n".format(sc.solve(list(existing_ids))))
    print("solve: {0}\n\n".format(sc.solve()))
    
    # Collecting results
    success, nD, fR, eR = sc.get_solved_results()
    print("pass criteria?\n {0}".format(success))

    trans_tol, rot_tol = sc.get_nodal_deformation_tol()

    max_trans, max_rot, max_trans_vid, max_rot_vid = sc.get_max_nodal_deformation()
    complaince = sc.get_compliance()
    
    print('max deformation: translation - {0} / tol {1}, at node #{2}'.format(max_trans, trans_tol, max_trans_vid))
    print('max deformation: rotation - {0} / tol {1}, at node #{2}'.format(max_rot, rot_tol, max_rot_vid))
    
    # visualize deformed structure
    orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)
#     print(orig_beam_shape)
    
    beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
#     print(beam_disp)
    meshcat_visualize_deformed(vis, beam_disp, orig_beam_shape, disc=disc, scale=scale)

solve: True


pass criteria?
 True
max deformation: translation - 3.55837915909e-07 / tol 0.001, at node #29
max deformation: rotation - 7.34087330948e-06 / tol 0.0523333333333, at node #2


# animate deformation for a construction sequence

In [45]:
import os
import time
cwd = os.getcwd()

json_path = os.path.join(cwd,"tower_3D.json")
# json_path = os.path.join(cwd,"sf-test_3-frame.json")
load_case_path = os.path.join(cwd,"tower_3D_load_case.json")

vis.delete()
disc = 10
exagg_ratio=1.0
time_step = 1.0

with sys_pipes():
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
    
#     sc.set_output_json(True)
#     sc.set_output_json_path(file_path = cwd, file_name = "sf-test_result.json")

#     ext_load = np.zeros([1,7])
#     ext_load[0,0] = 3
#     include_sw = False

#     ext_load, include_sw = cm.parse_load_case_from_json(load_case_path)
#     print('external load:')
#     print(ext_load)
#     print('include self weight : {0}'.format(include_sw))
#     sc.set_load(nodal_forces = ext_load)
#     sc.set_self_weight_load(include_sw)

    sc.set_self_weight_load(True)

    for i in range(0,24):
        vis.delete()
        existing_ids = list(range(0,i+1))
        sc.solve(existing_ids)

        orig_beam_shape = sc.get_original_shape(disc=disc, draw_full_shape=False)
        beam_disp = sc.get_deformed_shape(exagg_ratio=exagg_ratio, disc=disc)
        meshcat_visualize_deformed(vis, beam_disp, orig_beam_shape, disc=disc, scale=0.5)
        time.sleep(time_step)

# repetitive run test

In [49]:
import os
import time
cwd = os.getcwd()

json_path = os.path.join(cwd,'assembly_instances','extrusion', 'topopt-100_S1_03-14-2019_w_layer.json')
N = 10000

solve_partial = True
existing_ids = [125, 126, 115, 122, 111, 108, 23, 22, 98, 75, 64, 34, 61, 65, 59, 60, 39, 36, 44, 67]

check_result = False

## without reinit on everal `solve` call

In [50]:
with sys_pipes():
    sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
    
    st_time = time.time()
    for i in range(0,N):
        if solve_partial:
            sc.solve(existing_ids)
        else:
            sc.solve()
        
        if check_result:
            trans_tol, rot_tol = sc.get_nodal_deformation_tol()
            max_trans, max_rot, max_trans_vid, max_rot_vid = sc.get_max_nodal_deformation()
            complaince = sc.get_compliance()
            print('iter - {0}'.format(i))
            print('max deformation: translation : {0} / tol {1}, at node #{2}'.format(max_trans, trans_tol, max_trans_vid))
            print('max deformation: rotation : {0} / tol {1}, at node #{2}'.format(max_rot, rot_tol, max_rot_vid))
        
    print('avg time: {0} s'.format((time.time() - st_time) / N))

avg time: 0.000441094994545 s


## reinit on everal `solve` call

In [51]:
with sys_pipes():

    st_time = time.time()
    
    for i in range(0,N):
        sc = cm.stiffness_checker(json_file_path = json_path, verbose = False)
        
        existing_ids = list(range(0,10))
        if solve_partial:
            sc.solve(existing_ids)
        else:
            sc.solve()
        
        if check_result:
            trans_tol, rot_tol = sc.get_nodal_deformation_tol()
            max_trans, max_rot, max_trans_vid, max_rot_vid = sc.get_max_nodal_deformation()
            complaince = sc.get_compliance()
            print('iter - {0}'.format(i))
            print('max deformation: translation : {0} / tol {1}, at node #{2}'.format(max_trans, trans_tol, max_trans_vid))
            print('max deformation: rotation : {0} / tol {1}, at node #{2}'.format(max_rot, rot_tol, max_rot_vid))

    print('avg time: {0} s'.format((time.time() - st_time) / N))

avg time: 0.00134514820576 s
